In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

from sklearn.datasets import make_regression
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import PowerTransformer
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV


from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from worldcuptransformers import CustomTransformer

# Dataset Preparation
Read the dataset and clean datas. Remove outliers and transform features.
Create target with multi ouptu regressor problem.

In [93]:
# define constant & read the dataset
FOLDER = 'dataset/'
FILENAME = 'training_dataset.csv'

df = pd.read_csv(f'{FOLDER}{FILENAME}')
df.shape

(4371, 20)

In [94]:
df.columns.tolist()
df['matchdate']=pd.to_datetime(df['matchday'], format='%Y-%m-%d')
df

,Unnamed: 0,matchday,home_team,away_team,season,competition,correct_score,1N2_1,1N2_N,1N2_2,...,away_team_rank_FIFA,home_team_trend,away_team_trend,home_team_odd_trend,away_team_odd_trend,target,best_odd_won,corr_score_home,corr_score_away,matchdate
0,0,2002-05-31,France,Senegal,2002,world-cup,0:1,1.33,4.28,8.63,...,38.454545,-1.0,-1.0,-1.0000,-1.0000,2.0,False,0.999099,0.984973,2002-05-31
1,1,2002-06-01,Uruguay,Denmark,2002,world-cup,1:2,2.65,2.72,2.97,...,15.727273,-1.0,-1.0,-1.0000,4.3675,2.0,False,0.785714,0.988381,2002-06-01
2,2,2002-06-01,Ireland,Cameroon,2002,world-cup,1:1,3.33,2.90,2.28,...,19.090909,8.0,-1.0,2.1125,5.7825,1.0,False,0.881049,1.000000,2002-06-01
3,3,2002-06-01,Germany,Saudi Arabia,2002,world-cup,8:0,1.29,4.73,8.73,...,35.818182,10.0,10.0,2.5325,2.1125,0.0,True,0.989398,0.858464,2002-06-01
4,4,2002-06-02,Spain,Slovenia,2002,world-cup,3:1,1.42,4.10,7.57,...,30.909091,11.0,6.0,1.7250,2.2550,0.0,True,0.973360,0.952510,2002-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,4366,2021-07-03,Czech Republic,Denmark,2020,euro,1:2,3.94,3.08,2.13,...,10.666667,9.0,9.0,1.9425,2.0625,2.0,True,0.884615,0.988381,2021-07-03
4367,4367,2021-07-03,Ukraine,England,2020,euro,0:4,7.64,3.98,1.52,...,4.000000,12.0,12.0,1.8250,2.0300,2.0,True,0.956146,0.949018,2021-07-03
4368,4368,2021-07-06,Italy,Spain,2020,euro,2:1,2.50,3.14,3.10,...,6.000000,9.0,9.0,2.0100,1.9175,0.0,True,0.873981,0.973360,2021-07-06
4369,4369,2021-07-07,England,Denmark,2020,euro,2:1,1.75,3.44,5.58,...,10.666667,12.0,9.0,2.0675,2.1250,0.0,True,0.949018,0.988381,2021-07-07


In [95]:
df.columns.tolist()

['Unnamed: 0',
 'matchday',
 'home_team',
 'away_team',
 'season',
 'competition',
 'correct_score',
 '1N2_1',
 '1N2_N',
 '1N2_2',
 'home_team_rank_FIFA',
 'away_team_rank_FIFA',
 'home_team_trend',
 'away_team_trend',
 'home_team_odd_trend',
 'away_team_odd_trend',
 'target',
 'best_odd_won',
 'corr_score_home',
 'corr_score_away',
 'matchdate']

In [96]:
df.best_odd_won

0       False
1       False
2       False
3        True
4        True
        ...  
4366     True
4367     True
4368     True
4369     True
4370    False
Name: best_odd_won, Length: 4371, dtype: bool

In [97]:
features = ['1N2_1','1N2_N','1N2_2','home_team_rank_FIFA','away_team_rank_FIFA','season','home_team','away_team']
target = ['home_score','away_score']
for idx, r in df.iterrows():
    s = r.correct_score.split(':')
    df.loc[idx,'home_score']=int(s[0])
    df.loc[idx,'away_score']=int(s[1]) 
df.drop('correct_score', axis=1, inplace=True)    


In [98]:
trainset = df.query('competition != "world-cup" or (competition == "world-cup" and season < 2014)').drop('competition', axis=1)[features+target]
testset = df.query('competition == "world-cup" and (season <= 2014)').drop('competition', axis=1)[features+target]
unseenset = df.query('competition == "world-cup" and (season == 2018)').drop('competition', axis=1)[features+target]

wc2018 =df.query('competition == "world-cup" and (season == 2018)')



In [99]:
trainset.shape, testset.shape, unseenset.shape

((4291, 10), (193, 10), (64, 10))

In [100]:
trainset.sample(7)

,1N2_1,1N2_N,1N2_2,home_team_rank_FIFA,away_team_rank_FIFA,season,home_team,away_team,home_score,away_score
114,2.82,3.09,2.29,8.666667,8.000000,2005,USA,England,1.0,2.0
2660,1.63,3.66,5.34,29.416667,59.750000,2015,Ecuador,Panama,4.0,0.0
256,1.09,7.00,16.83,27.181818,158.454545,2006,Poland,Faroe Islands,4.0,0.0
1459,1.65,3.74,4.62,94.818182,72.636364,2010,Kuwait,Zambia,4.0,0.0
790,3.57,3.24,2.01,40.916667,3.750000,2008,Switzerland,Germany,0.0,4.0
2211,1.12,8.23,18.85,4.750000,76.916667,2014,Spain,Bolivia,2.0,0.0
1865,2.67,3.13,2.54,49.250000,50.250000,2012,Tunisia,Iran,2.0,2.0


In [101]:
#trainset.dtypes, testset.dtypes, unseenset.dtypes

In [102]:
from sklearn.preprocessing import StandardScaler

In [103]:
train_X = trainset.drop(['home_score','away_score'],axis=1).values
train_y = trainset[['home_score', 'away_score']].values


test_X = testset.drop(['home_score','away_score'],axis=1).values
test_y = testset[['home_score', 'away_score']].values


unseen_X = unseenset.drop(['home_score','away_score'],axis=1).values



In [104]:
trainset.drop(['home_score','away_score'], axis=1).columns
              

Index(['1N2_1', '1N2_N', '1N2_2', 'home_team_rank_FIFA', 'away_team_rank_FIFA',
       'season', 'home_team', 'away_team'],
      dtype='object')

In [105]:
regressor = KNeighborsRegressor()


import warnings
warnings.filterwarnings("ignore")
from sklearn.exceptions import FitFailedWarning 

In [106]:
trainset.columns

Index(['1N2_1', '1N2_N', '1N2_2', 'home_team_rank_FIFA', 'away_team_rank_FIFA',
       'season', 'home_team', 'away_team', 'home_score', 'away_score'],
      dtype='object')

In [107]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

regressor = DecisionTreeRegressor(max_features='log2',criterion='friedman_mse')
#regressor = MultiOutputRegressor(GradientBoostingRegressor(random_state=678, learning_rate=0.95795, criterion='squared_error'))
#regressor = KNeighborsRegressor(n_neighbors=15)
#regressor =  LinearRegression()

model = LinearSVR()
# define the chained multioutput wrapper model
#regressor = RegressorChain(model)


train_X = trainset.drop(['home_score','away_score'],axis=1)
train_y = trainset[['home_score', 'away_score']]

test_X = testset.drop(['home_score','away_score'],axis=1)
test_y = testset[['home_score', 'away_score']]

numeric_features = ['1N2_1', '1N2_N', '1N2_2', 'away_team_rank_FIFA', 'home_team_rank_FIFA',
       'season']
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ['home_team','away_team']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


clf = Pipeline(
    steps=[
        ('experimental_trans', CustomTransformer('1N2_1')) ,       
        ("preprocessor", preprocessor), 

        ("classifier", regressor)
        ]
)


clf.fit(train_X, train_y)
print("model score: %.3f" % clf.score(test_X, test_y))


model score: 0.697


In [108]:

import joblib

#save your model or results
joblib.dump(clf,'models/multi-regressor-score.pkl')

#load your model for further usage
model = joblib.load("models/multi-regressor-score.pkl")

In [109]:
clf

Pipeline(steps=[('experimental_trans', CustomTransformer(feature_name='1N2_1')),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['1N2_1', '1N2_N', '1N2_2',
                                                   'away_team_rank_FIFA',
                                                   'home_team_rank_FIFA',
                                                   'season']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['home_team',
                                                   'away_team'])])),
                ('classifier',
                 DecisionTreeRegressor(criterion='friedman_mse',
                                       max_features='log2'))])

In [110]:
model.predict(test_X.iloc[0:3,:])

array([[0., 1.],
       [1., 2.],
       [1., 1.]])

In [111]:
grid = clf

In [112]:
trainset.columns

Index(['1N2_1', '1N2_N', '1N2_2', 'home_team_rank_FIFA', 'away_team_rank_FIFA',
       'season', 'home_team', 'away_team', 'home_score', 'away_score'],
      dtype='object')

In [113]:
train_X.dtypes

1N2_1                  float64
1N2_N                  float64
1N2_2                  float64
home_team_rank_FIFA    float64
away_team_rank_FIFA    float64
season                   int64
home_team               object
away_team               object
dtype: object

In [114]:
test_X

,1N2_1,1N2_N,1N2_2,home_team_rank_FIFA,away_team_rank_FIFA,season,home_team,away_team
0,1.33,4.28,8.63,1.818182,38.454545,2002,France,Senegal
1,2.65,2.72,2.97,23.545455,15.727273,2002,Uruguay,Denmark
2,3.33,2.90,2.28,54.727273,19.090909,2002,Ireland,Cameroon
3,1.29,4.73,8.73,7.545455,35.818182,2002,Germany,Saudi Arabia
4,1.42,4.10,7.57,5.090909,30.909091,2002,Spain,Slovenia
...,...,...,...,...,...,...,...,...
2283,1.52,4.13,7.02,8.333333,24.250000,2014,Netherlands,Costa Rica
2284,2.64,3.15,2.91,6.583333,1.500000,2014,Brazil,Germany
2285,3.03,3.20,2.52,8.333333,3.250000,2014,Netherlands,Argentina
2286,2.16,3.67,3.28,6.583333,8.333333,2014,Brazil,Netherlands


In [115]:
model.predict(test_X.iloc[0:1,:])

array([[0., 1.]])

In [116]:
#pip install xgboost